In [ ]:
!pip -q install datasets
!pip -q install transformers
!pip -q install simpletransformers
!pip -q install deepcut
!pip -q install pyicu
# !pip -q install tqdm==4.47.0
!pip install git+https://github.com/PyThaiNLP/pythainlp.git@dev

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tinydb 4.5.1 requires typing-extensions<4.0.0,>=3.10.0; python_version <= "3.7", but you have typing-extensions 3.7.4.3 which is incompatible.
  Cloning https://github.com/PyThaiNLP/pythainlp.git (to revision dev) to /tmp/pip-req-build-thg_hccm
  Running command git clone -q https://github.com/PyThaiNLP/pythainlp.git /tmp/pip-req-build-thg_hccm
  Using cached typing_extensions-3.10.0.0-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires typing-e

In [ ]:
import torch

In [ ]:
import numpy as np
import pandas as pd
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

from pythainlp import sent_tokenize, Tokenizer
from pythainlp.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
import re
from collections import OrderedDict

In [ ]:
import pythainlp
pythainlp.__version__
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sun Jul 25 17:48:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip "/content/drive/MyDrive/Hackathon/wiki-documents-nsc.zip"

Streaming output truncated to the last 5000 lines.
  inflating: wiki-documents-nsc/92021.txt  
  inflating: wiki-documents-nsc/92022.txt  
  inflating: wiki-documents-nsc/92025.txt  
  inflating: wiki-documents-nsc/92027.txt  
  inflating: wiki-documents-nsc/92028.txt  
  inflating: wiki-documents-nsc/92030.txt  
  inflating: wiki-documents-nsc/92035.txt  
  inflating: wiki-documents-nsc/92039.txt  
  inflating: wiki-documents-nsc/92059.txt  
  inflating: wiki-documents-nsc/92077.txt  
  inflating: wiki-documents-nsc/92078.txt  
  inflating: wiki-documents-nsc/92080.txt  
  inflating: wiki-documents-nsc/92082.txt  
  inflating: wiki-documents-nsc/92089.txt  
  inflating: wiki-documents-nsc/92098.txt  
  inflating: wiki-documents-nsc/92099.txt  
  inflating: wiki-documents-nsc/92101.txt  
  inflating: wiki-documents-nsc/92102.txt  
  inflating: wiki-documents-nsc/92103.txt  
  inflating: wiki-documents-nsc/92104.txt  
  inflating: wiki-documents-nsc/92105.txt  
  inflating: wiki-documen

In [ ]:
qa_train_df = pd.read_csv("/content/drive/MyDrive/Hackathon/ThaiQACorpus-DevelopmentDataset.csv", encoding='utf8')



In [ ]:
qa_small_test_df = pd.read_csv("/content/drive/MyDrive/Hackathon/Test.csv", encoding='utf8')

In [ ]:
qa_train_df.head()

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id
0,1,1,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,ฮิกกิ้นส์,529.0,538.0,115035
1,2,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,เมชตา,139.0,144.0,376583
2,3,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,ลมสุริยะ,641.0,649.0,376583
3,4,1,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,ปี 1933,304.0,311.0,547560
4,5,1,ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบด...,ประธานาธิบดีวิลเลียม เอช. ทัฟต์,231.0,262.0,193032


# **Prepare Data**

In [ ]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F000-\U0001FFFF"
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
def replaceSpace(text):
  replaceWord = [['“ ','“']
                 ,[' ”','”']
                 ,['" ','"']
                 ,[' "','"']
                 ,['   ',' ']
                 ,['  ',' ']
                #  ,['# ','#']
                 ,['http :// bit . ly /','http://bit.ly/']                 
                 ]

  for i in range(len(replaceWord)):
    text = text.replace(replaceWord[i][0],replaceWord[i][1])

  return text

In [ ]:
def remove(text):
  text2 = deEmojify(text).replace("\n.", "")
  text2 = text2.replace("\n", "")
  text2 = text2.replace("!", " ")
  text2 = text2.replace("\ufeff", "")
  text2 = text2.replace("➡️", "")
  #text2 = text2.replace("”", "\"")
  return text2

In [ ]:
def readfile(path):
    f = open("/content/wiki-documents-nsc/"+str(path)+'.txt', "r")
    text = f.read()
    text = remove(text)
    return text

In [ ]:
def GetStartURL(text):
  keyText = [
             ['https://bit.ly/SCGscholarship-apply',35]
             ,['http://bit.ly/SCGDigitalOffice',30]
             ,['http://bit.ly/SCGscholarship',28]
             ,['http://bit.ly/',21]

                  ]

  for i in range(len(keyText)):
    if keyText[i][0] in text:      
      return [text.index(keyText[i][0]),keyText[i][1],len(keyText[i][0])]

  return [-1,0,0]

def concatURL(text,stopAtLenght=False):
  newtext = text
  indexStartURL,URLLenght,KeyLenght = GetStartURL(text)
  try:
    if indexStartURL>-1:
      for indexWord in range(indexStartURL+KeyLenght,(indexStartURL+URLLenght)*2,1):
        tmpURL = text[indexStartURL:indexWord]    
        tmpURL = tmpURL.replace(' ','')
        if len(tmpURL) == URLLenght:
          tmp=text[indexStartURL:indexWord]
          nextTmp = text[indexWord:indexWord+1]
          if nextTmp==".":
            tmp=text[indexStartURL:indexWord+1]
          
          newtext = newtext.replace(tmp,tmpURL)          
          if stopAtLenght==True:
            newtext = newtext[0:URLLenght]
          return newtext        
    return newtext
  except:
    return text

In [ ]:
def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

In [ ]:
from pythainlp.tokenize import word_tokenize

In [ ]:
qa_train_df["context_text"] = qa_train_df['article_id'].apply(readfile)
qa_train_df.head()

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id,context_text
0,1,1,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,ฮิกกิ้นส์,529.0,538.0,115035,"<doc id=""115035"" url=""https://th.wikipedia.org..."
1,2,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,เมชตา,139.0,144.0,376583,"<doc id=""376583"" url=""https://th.wikipedia.org..."
2,3,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,ลมสุริยะ,641.0,649.0,376583,"<doc id=""376583"" url=""https://th.wikipedia.org..."
3,4,1,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,ปี 1933,304.0,311.0,547560,"<doc id=""547560"" url=""https://th.wikipedia.org..."
4,5,1,ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบด...,ประธานาธิบดีวิลเลียม เอช. ทัฟต์,231.0,262.0,193032,"<doc id=""193032"" url=""https://th.wikipedia.org..."


In [ ]:
from lxml import etree
parser = etree.XMLParser(recover=True)
tree = etree.fromstring(qa_train_df["context_text"][0], parser=parser)
print(tree.tag)
print(tree.attrib)
print(tree.text)

doc
{'id': '115035', 'url': 'https://th.wikipedia.org/wiki?curid=115035', 'title': 'เบนจี้'}
เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู้ ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบท และกำกับโดย โจ แคมป์ ในช่วงทศวรรษ 1970 ถึง 1980 ภาพยนตร์เรื่องแรกในชุด ใช้ชื่อเรื่องว่า เบนจี้ เช่นเดียวกับตัวละคร ถ่ายทำที่เมืองดัลลัส รัฐเทกซัส ฉายครั้งแรกในปี พ.ศ. 2517 ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์ และได้รางวัลลูกโลกทองคำ สาขาเพลงประกอบยอดเยี่ยม จากเพลง Benji's Theme (I Feel Love) ร้องโดย ชาร์ลี ริช หมาที่แสดงเป็นเบนจี้ตัวแรก ชื่อว่า ฮิกกิ้นส์ (พ.ศ. 2502 - พ.ศ. 2518) มีอายุถึง 15 ปีแล้วในขณะแสดง หลังจากภาพยนตร์ออกฉายได้ไม่นาน มันก็ตายในปี พ.ศ. 2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์. - พ.ศ. 2517, Benji (ภาพยนตร์) - พ.ศ. 2520, For the Love of Benji (ภาพยนตร์) - พ.ศ. 2521, Benji's Very Own Christmas Story (ภาพยนตร์โทรทัศน์) - พ.ศ. 2523, Oh Heavenly Dog (ภาพยนตร์) - พ.ศ. 2523, Benji at Work (ภาพยนตร์โทรทัศน์) - พ.ศ. 2524, Benji Takes a Dive at Marineland (ภาพยนตร์โทรทัศน์) - พ.ศ. 2526, Benji

In [ ]:
def extract_text(body):
  try:
    tree = etree.fromstring(body, parser=parser)
    return tree.text
  except:
    return body

In [ ]:
qa_train_df["context_text"] = qa_train_df["context_text"].apply(extract_text)

In [ ]:
qa_train_df.head(5)

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id,context_text
0,1,1,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,ฮิกกิ้นส์,529.0,538.0,115035,เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู...
1,2,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,เมชตา,139.0,144.0,376583,ลูนา 1 ลูนา 1 (อี-1 ซีรีส์) ซึ่งในขณะนั้นรู้จั...
2,3,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,ลมสุริยะ,641.0,649.0,376583,ลูนา 1 ลูนา 1 (อี-1 ซีรีส์) ซึ่งในขณะนั้นรู้จั...
3,4,1,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,ปี 1933,304.0,311.0,547560,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อ กระทรวงโฆ...
4,5,1,ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบด...,ประธานาธิบดีวิลเลียม เอช. ทัฟต์,231.0,262.0,193032,ห้องทำงานรูปไข่ ห้องทำงานรูปไข่ (Oval Office) ...


In [ ]:
questions = qa_train_df["question"].apply(remove)

In [ ]:
qa_train_df["context_text"][0]

"เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู้ ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบท และกำกับโดย โจ แคมป์ ในช่วงทศวรรษ 1970 ถึง 1980 ภาพยนตร์เรื่องแรกในชุด ใช้ชื่อเรื่องว่า เบนจี้ เช่นเดียวกับตัวละคร ถ่ายทำที่เมืองดัลลัส รัฐเทกซัส ฉายครั้งแรกในปี พ.ศ. 2517 ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์ และได้รางวัลลูกโลกทองคำ สาขาเพลงประกอบยอดเยี่ยม จากเพลง Benji's Theme (I Feel Love) ร้องโดย ชาร์ลี ริช หมาที่แสดงเป็นเบนจี้ตัวแรก ชื่อว่า ฮิกกิ้นส์ (พ.ศ. 2502 - พ.ศ. 2518) มีอายุถึง 15 ปีแล้วในขณะแสดง หลังจากภาพยนตร์ออกฉายได้ไม่นาน มันก็ตายในปี พ.ศ. 2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์. - พ.ศ. 2517, Benji (ภาพยนตร์) - พ.ศ. 2520, For the Love of Benji (ภาพยนตร์) - พ.ศ. 2521, Benji's Very Own Christmas Story (ภาพยนตร์โทรทัศน์) - พ.ศ. 2523, Oh Heavenly Dog (ภาพยนตร์) - พ.ศ. 2523, Benji at Work (ภาพยนตร์โทรทัศน์) - พ.ศ. 2524, Benji Takes a Dive at Marineland (ภาพยนตร์โทรทัศน์) - พ.ศ. 2526, Benji, Zax  the Alien Prince (ภาพยนตร์ซีรีส์) - พ.ศ. 2530, Benji the Hunted (ภาพยนตร์) - พ.ศ. 254

In [ ]:
from pythainlp.tokenize import word_tokenize
qa_train_df["context_text_wordsegged"] = qa_train_df["context_text"].apply(lambda x: ' '.join(word_tokenize(x)))
#qa_train_df["context_text_wordsegged"] = qa_train_df["context_text_wordsegged"].apply(lambda x: ' '.join(replaceSpace(x)))


In [ ]:
qa_train_df["context_text_wordsegged"][10]

In [ ]:
qa_train_df["context_text_wordsegged"] = qa_train_df["context_text_wordsegged"].apply(lambda x: replaceSpace(x))

In [ ]:
qa_train_df["context_text_wordsegged"][1]

'ลู นา 1 ลู นา 1 ( อี - 1 ซีรีส์ ) ซึ่ง ในขณะนั้น รู้จัก กัน ใน ชื่อ เม ช ตา (; "ความฝัน") เป็น ยานอวกาศ ลำ แรก ที่ เดินทาง ไป ถึง บริเวณ ใกล้เคียง ของ ดวงจันทร์ และ เป็น ยานอวกาศ ลำ แรก ใน โครงการ ลู นา ของ โซเวียต ที่ สามารถ ปล่อย ขึ้นไป ใน ทิศ ทางเดียว กับ ดวงจันทร์ ได้ สำเร็จ ขณะ เดิน ทางผ่าน แถบ รังสี แวน แอล เลน ชั้นนอก เครื่องมือ ตรวจวัด รังสี ของ ยาน สามารถ ตรวจจับ อนุภาค พลังงาน สูง ปริมาณ เล็กน้อย ที่ มี อยู่ ใน วงแหวน ชั้นนอก ค่าที่ ได้ จาก การตรวจวัด ดังกล่าว ทำ ให้ได้ ข้อมูล ใหม่ เกี่ยวกับ แถบ รังสี ของ โลก และ อวกาศ ปรากฏ ว่า ดวงจันทร์ ไม่ มี สนามแม่เหล็ก ที่ สามารถ ตรวจ พบ ได้ นอกจากนี้ ลู นา 1 ยัง เป็นครั้งแรก ที่ มี การ สังเกต และ ตรวจวัด ลม สุริยะ โดยตรง ความ เข้มข้น ของ พลาสมา ที่ ถูก เปลี่ยนเป็น ไอออน ดังกล่าว วัด ปริมาณ ได้ ราว 700 อนุภาค ต่อ ลบ.ซม. ที่ ระดับความสูง 20,000 - 25,000 กิโลเมตร และ 300 ถึง 400 อนุภาค ต่อ ลบ.ซม. ที่ ระดับความสูง 100,000 - 150,000 กิโลเมตร ลู นา 1 ยัง ได้ มี การ ติดต่อ ทาง วิทยุ เป็นครั้งแรก จาก ระยะทาง ครึ่ง ล้าน กิโลเมตร จาก พื้นโลก'

In [ ]:
qa_train_df["context_text_wordsegged"] = qa_train_df["context_text_wordsegged"].apply(lambda x: replaceSpace(x))

In [ ]:
qa_train_df["question_wordsegged"] = qa_train_df["question"].apply(lambda x: ' '.join(word_tokenize(x)))

In [ ]:
qa_train_df["question_wordsegged"] = qa_train_df["question_wordsegged"].apply(lambda x: replaceSpace(x))

In [ ]:
qa_train_df["answers_wordsegged"] = qa_train_df["answer"].apply(lambda x: ' '.join(word_tokenize(x)))
qa_train_df["answers_wordsegged"] = qa_train_df["answers_wordsegged"].apply(lambda x: replaceSpace(x))
qa_train_df["answers_wordsegged"] = qa_train_df["answers_wordsegged"].apply(lambda x: concatURL(x))

In [ ]:
qa_train_df["context_text_wordsegged"][5]

'เครื่อง อิสริยาภรณ์ ดารา มหา ปุ ตรา เครื่อง อิสริยาภรณ์ ดารา มหา ปุ ตรา ( บิน ตัง มหา ปุ ตรา ) เป็น เครื่อง อิสริยาภรณ์ ที่ มี ลำดับ เกียรติ เป็น อันดับ ที่ 2 ของ ประเทศ อินโดนีเซีย รอง จาก เครื่อง อิสริยาภรณ์ ดารา แห่ง สาธารณรัฐ อินโดนีเซีย เครื่อง อิสริยาภรณ์ ตระกูล นี้ มอบ โดย รัฐบาล โดย มี ประธานาธิบดี แห่ง สาธารณรัฐ อินโดนีเซีย เป็น ประธาน แห่ง เครื่อง อิสริยาภรณ์ เพื่อ เป็น บำเหน็จ ความชอบ ใน การ ปฏิบัติหน้าที่ เพื่อ มาตุภูมิ อย่าง โดดเด่น เป็นพิเศษ ใน พื้นที่ ที่อยู่ นอกขอบเขต ของ ทหาร มี ทั้งหมด 5 ชั้น ลำดับชั้น ลำดับชั้น . - ชั้น ที่ 1 บิน ตัง มหา ปุ ตรา อธิ ปุ รณา ( Bintang Mahaputra Adipurna ) - ชั้น ที่ 2 บิน ตัง มหา ปุ ตรา อธิ ประ ธา นา ( Bintang Mahaputra Adipradana ) - ชั้น ที่ 3 บิน ตัง มหา ปุ ตรา อุ ต มา ( Bintang Mahaputra Utama ) - ชั้น ที่ 4 บิน ตัง มหา ปุ ตรา ประ ถ มา ( Bintang Mahaputra Pratama ) - ชั้น ที่ 5 บิน ตัง มหา ปุ ตรา นารา ร์ ยา ( Bintang Mahaputra Nararya )'

In [ ]:
qa_train_df["question_wordsegged"][5]

'เครื่อง อิสริยาภรณ์ ดารา มหา ปุ ตรา เป็น เครื่อง อิสริยาภรณ์ ของ ประเทศ อินโดนีเซีย มี ลำดับ เกียรติ เป็น อันดับ ที่ เท่าไร'

In [ ]:
qa_train_df["answers_wordsegged"][5]

'อันดับ ที่ 2'

In [ ]:
answer_starts = []
for idx, row in qa_train_df.iterrows():
  try:
    answer_start = row["context_text_wordsegged"].index(row["answers_wordsegged"])
  except:
    answer_start = -1
  answer_starts.append(answer_start)

In [ ]:
qa_train_df["answer_starts"] = answer_starts

In [ ]:
qa_train_df.tail()

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id,context_text,context_text_wordsegged,question_wordsegged,answers_wordsegged,answer_starts
16995,16996,2,ซังกริอา เป็นเครื่องดื่มประเภทหนึ่งจากสเปนและอ...,ไม่ใช่,NaN,NaN,713240,ซังกริอา ซังกริอา (; ) เป็นเครื่องดื่มประเภทหน...,ซัง ก ริ อา ซัง ก ริ อา (; ) เป็น เครื่องดื่ม ...,ซัง ก ริ อา เป็น เครื่องดื่ม ประเภท หนึ่ง จาก ...,ไม่ ใช่,-1
16996,16997,2,ซังกริอา ประกอบด้วยไวน์ ผลไม้หั่น สารให้ความหว...,ไม่ใช่,NaN,NaN,713240,ซังกริอา ซังกริอา (; ) เป็นเครื่องดื่มประเภทหน...,ซัง ก ริ อา ซัง ก ริ อา (; ) เป็น เครื่องดื่ม ...,ซัง ก ริ อา ประกอบด้วย ไวน์ ผลไม้ หั่น สาร ให้...,ไม่ ใช่,-1
16997,16998,2,หมู่เกาะกระเป็นหมู่เกาะเดียวของจังหวัดชุมพร ซึ...,ไม่ใช่,NaN,NaN,396798,เกาะกระ เกาะกระ หรือ หมู่เกาะกระ อยู่ในพื้นที่...,เกาะ กระ เกาะ กระ หรือ หมู่เกาะ กระ อยู่ ใน พื...,หมู่เกาะ กระ เป็น หมู่เกาะ เดียว ของ จังหวัด ช...,ไม่ ใช่,-1
16998,16999,2,เกาะกระ หรือ หมู่เกาะกระ ประกอบด้วย เกาะขนาดเล...,ไม่ใช่,NaN,NaN,396798,เกาะกระ เกาะกระ หรือ หมู่เกาะกระ อยู่ในพื้นที่...,เกาะ กระ เกาะ กระ หรือ หมู่เกาะ กระ อยู่ ใน พื...,เกาะ กระ หรือ หมู่เกาะ กระ ประกอบด้วย เกาะ ขนา...,ไม่ ใช่,-1
16999,17000,2,กาแฟขี้ช้างเป็นหนึ่งในกาแฟที่ถูกหรือแพงขึ้นอยู...,ไม่ใช่,NaN,NaN,938948,แบล็ค ไอวอรี่ คอฟฟี่การวางขาย การวางขาย. กาแฟข...,แบล็ค ไอ วอ รี่ คอ ฟ ฟี่ การ วางขาย การ วางขาย...,กาแฟ ขี้ ช้าง เป็นหนึ่ง ใน กาแฟ ที่ ถูก หรือ แ...,ไม่ ใช่,-1


In [ ]:
qa_train, not_use = train_test_split(qa_train_df, test_size=0.77,random_state=0)

In [ ]:
qa_train, qa_test = train_test_split(qa_train, test_size=0.33,random_state=0)

In [ ]:
print(len(qa_test))
print(len(qa_train))
print(len(qa_train_df))

1291
2619
17000


In [ ]:
qa_train_df

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id,context_text,context_text_wordsegged,question_wordsegged,answers_wordsegged,answer_starts
0,1,1,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,ฮิกกิ้นส์,529.0,538.0,115035,เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู...,เบน จี้ เบน จี้ () เป็น ชื่อ ตัวละคร หมา พันทา...,สุนัข ตัว แรก รับบท เป็น เบน จี้ ใน ภาพยนตร์ เ...,ฮิ กกิ้นส์,510
1,2,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,เมชตา,139.0,144.0,376583,ลูนา 1 ลูนา 1 (อี-1 ซีรีส์) ซึ่งในขณะนั้นรู้จั...,ลู นา 1 ลู นา 1 ( อี - 1 ซีรีส์ ) ซึ่ง ในขณะนั...,ลู นา 1 เป็น ยานอวกาศ ลำ แรก ใน โครงการ ลู นา ...,เม ช ตา,68
2,3,1,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,ลมสุริยะ,641.0,649.0,376583,ลูนา 1 ลูนา 1 (อี-1 ซีรีส์) ซึ่งในขณะนั้นรู้จั...,ลู นา 1 ลู นา 1 ( อี - 1 ซีรีส์ ) ซึ่ง ในขณะนั...,ลู นา 1 เป็น ยานอวกาศ ลำ แรก ใน โครงการ ลู นา ...,ลม สุริยะ,663
3,4,1,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อของนาซีเยอ...,ปี 1933,304.0,311.0,547560,กระทรวงโฆษณาแถลงข่าวและโฆษณาชวนเชื่อ กระทรวงโฆ...,กระทรวง โฆษณา แถลงข่าว และ โฆษณาชวนเชื่อ กระทร...,กระทรวง โฆษณา แถลงข่าว และ โฆษณาชวนเชื่อ ของ น...,ปี 1933,219
4,5,1,ห้องทำงานรูปไข่ สร้างขึ้นในสมัยอดีตประธานาธิบด...,ประธานาธิบดีวิลเลียม เอช. ทัฟต์,231.0,262.0,193032,ห้องทำงานรูปไข่ ห้องทำงานรูปไข่ (Oval Office) ...,ห้องทำงาน รูปไข่ ห้องทำงาน รูปไข่ ( Oval Offic...,ห้องทำงาน รูปไข่ สร้าง ขึ้น ใน สมัย อดีต ประธา...,ประธานาธิบดี วิลเลียม เอช . ทัฟต์,157
...,...,...,...,...,...,...,...,...,...,...,...,...
16995,16996,2,ซังกริอา เป็นเครื่องดื่มประเภทหนึ่งจากสเปนและอ...,ไม่ใช่,NaN,NaN,713240,ซังกริอา ซังกริอา (; ) เป็นเครื่องดื่มประเภทหน...,ซัง ก ริ อา ซัง ก ริ อา (; ) เป็น เครื่องดื่ม ...,ซัง ก ริ อา เป็น เครื่องดื่ม ประเภท หนึ่ง จาก ...,ไม่ ใช่,-1
16996,16997,2,ซังกริอา ประกอบด้วยไวน์ ผลไม้หั่น สารให้ความหว...,ไม่ใช่,NaN,NaN,713240,ซังกริอา ซังกริอา (; ) เป็นเครื่องดื่มประเภทหน...,ซัง ก ริ อา ซัง ก ริ อา (; ) เป็น เครื่องดื่ม ...,ซัง ก ริ อา ประกอบด้วย ไวน์ ผลไม้ หั่น สาร ให้...,ไม่ ใช่,-1
16997,16998,2,หมู่เกาะกระเป็นหมู่เกาะเดียวของจังหวัดชุมพร ซึ...,ไม่ใช่,NaN,NaN,396798,เกาะกระ เกาะกระ หรือ หมู่เกาะกระ อยู่ในพื้นที่...,เกาะ กระ เกาะ กระ หรือ หมู่เกาะ กระ อยู่ ใน พื...,หมู่เกาะ กระ เป็น หมู่เกาะ เดียว ของ จังหวัด ช...,ไม่ ใช่,-1
16998,16999,2,เกาะกระ หรือ หมู่เกาะกระ ประกอบด้วย เกาะขนาดเล...,ไม่ใช่,NaN,NaN,396798,เกาะกระ เกาะกระ หรือ หมู่เกาะกระ อยู่ในพื้นที่...,เกาะ กระ เกาะ กระ หรือ หมู่เกาะ กระ อยู่ ใน พื...,เกาะ กระ หรือ หมู่เกาะ กระ ประกอบด้วย เกาะ ขนา...,ไม่ ใช่,-1


In [ ]:
qa_test

,question_id,question_type,question,answer,answer_begin_position,answer_end_position,article_id,context_text,context_text_wordsegged,question_wordsegged,answers_wordsegged,answer_starts
3873,3874,1,เครื่องอิสริยาภรณ์ซามูซนิสท์ของประเทศยูเครน ถู...,ลีโอนิส คุชมา,236.0,249.0,906038,เครื่องอิสริยาภรณ์ซามูซนิสท์ เครื่องอิสริยาภรณ...,เครื่อง อิสริยาภรณ์ ซา มูซ นิ สท์ เครื่อง อิสร...,เครื่อง อิสริยาภรณ์ ซา มูซ นิ สท์ ของ ประเทศ ย...,ลีโอ นิ ส คุ ชมา,152
3625,3626,1,หมู่เกาะมาเดราในประเทศโปรตุเกสตั้งอยู่ในมหาสมุ...,มหาสมุทรแอตแลนติก,149.0,166.0,93102,มาเดรา มาเดรา () เป็นหมู่เกาะของประเทศโปรตุเกส...,มา เดรา มา เดรา () เป็น หมู่เกาะ ของ ประเทศ โป...,หมู่เกาะ มา เดรา ในประเทศ โปรตุเกส ตั้งอยู่ ใน...,มหาสมุทร แอตแลนติก,79
3028,3029,1,พ่อขุนรามคำแหงทรงประดิษฐ์อักษรไทยขึ้นเมื่อปี พ...,1826,1210.0,1214.0,2088,ภาษาไทย ภาษาไทย เป็นภาษาราชการของประเทศไทย ภาษ...,ภาษาไทย ภาษาไทย เป็น ภาษาราชการ ของ ประเทศ ไทย...,พ่อขุน รามคำแหง ทรง ประดิษฐ์ อักษร ไทย ขึ้น เม...,1826,1320
13814,13815,1,คิม จง-แด เป็นนักร้องนักแต่งเพลงชาวเกาหลีใต้ที...,เฉิน,181.0,185.0,894563,เฉิน (นักร้อง) คิม จง-แด (เกิดวันที่ 21 กันยาย...,เฉิน ( นักร้อง ) คิม จง - แด ( เกิด วันที่ 21 ...,คิม จง - แด เป็น นักร้อง นักแต่งเพลง ชาว เกาหล...,เฉิน,0
15398,15399,2,X-เม็น รุ่น 1 เป็นปัจฉิมบทของภาพยนตร์ชุด X-เม็...,ไม่ใช่,NaN,NaN,376252,X-เม็น รุ่น 1 X-เม็น รุ่น 1 () เป็นภาพยนตร์ปฐม...,X- เม็น รุ่น 1 X- เม็น รุ่น 1 () เป็น ภาพยนตร์...,X- เม็น รุ่น 1 เป็น ปัจฉิม บท ของ ภาพยนตร์ ชุด...,ไม่ ใช่,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
12830,12831,1,ปริญญา เจริญผล อดีตแชมเปียนมวยไทยใช้ชื่อในการช...,ปริญญา เกียรติบุษบา,269.0,288.0,317110,ปริญญา เจริญผล ปริญญา เจริญผล หรือเป็นที่รู้จั...,ปริญญา เจริญ ผล ปริญญา เจริญ ผล หรือ เป็นที่รู...,ปริญญา เจริญ ผล อดีต แช ม เปีย นม วย ไทย ใช้ ช...,ปริญญา เกียรติ บุษบา,203
7488,7489,1,ชาวซีดังในเวียดนามใช้โสมเวียดนามเป็นยาสมุนไพรต...,บำรุงร่างกายให้แข็งแรง,556.0,578.0,650450,โสมเวียดนาม โสมเวียดนาม เป็นพืชในวงศ์ Araliace...,โสม เวียดนาม โสม เวียดนาม เป็น พืช ใน วงศ์ Ara...,ชาว ซี ดัง ใน เวียดนาม ใช้ โสม เวียดนาม เป็น ย...,บำรุง ร่างกาย ให้ แข็งแรง,556
11275,11276,1,บุญมา วงศ์สวรรค์เคยเป็นอดีตรัฐมนตรีว่าการกระทร...,นายสัญญา ธรรมศักดิ์,193.0,212.0,711987,บุญมา วงศ์สวรรค์ บุญมา วงศ์สวรรค์ ราชบัณฑิตกิต...,บุญ มา วงศ์ สวรรค์ บุญ มา วงศ์ สวรรค์ ราชบัณฑิ...,บุญ มา วงศ์ สวรรค์ เคย เป็นอดีต รัฐมนตรี ว่ากา...,นาย สัญญา ธรรม ศักดิ์,111
4304,4305,1,บ้านของบิล เกตส์ ที่เมืองเมดินา รัฐวอชิงตัน มี...,"66,000",243.0,249.0,340501,บ้านของบิล เกตส์ บ้านของบิล เกตส์ เป็นคฤหาสน์ข...,บ้าน ของ บิล เกตส์ บ้าน ของ บิล เกตส์ เป็น คฤห...,บ้าน ของ บิล เกตส์ ที่ เมือง เมดินา รัฐ วอชิงต...,"66,000",174


In [ ]:
test_data = []
for idx, row in qa_test.iterrows():
  doc = {}
  doc["context"] = row["context_text_wordsegged"]
  doc["qas"] = [{"id":idx, "is_impossible":row["answer_starts"]<0,"question":row["question_wordsegged"],"answers":[{"text":row["answers_wordsegged"], "answer_start":row["answer_starts"]}]}]
  test_data.append(doc)

In [ ]:
train_data = []
for idx, row in qa_train.iterrows():
  doc = {}
  doc["context"] = row["context_text_wordsegged"]
  doc["qas"] = [{"id":idx, "is_impossible":row["answer_starts"]<0,"question":row["question_wordsegged"],"answers":[{"text":row["answers_wordsegged"], "answer_start":row["answer_starts"]}]}]
  train_data.append(doc)

In [ ]:
test_data

In [ ]:
print(type(train_data))

# **MODEL**

In [ ]:
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install simpletransformers

In [ ]:
!pip -q install pythainlp

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from google.colab import widgets

In [ ]:
pretrained = "deepset/xlm-roberta-large-squad2"#@param ["monsoon-nlp/bert-base-thai", "bert-base-multilingual-uncased", "Geotrend/bert-base-th-cased","puri/puri-thai-albert-cased-v1","monsoon-nlp/thai-electra","deepset/xlm-roberta-large-squad2"] 

In [ ]:
model_name = pretrained

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:

# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = False
model_args.num_train_epochs = 3
model_args.overwrite_output_dir = True
model_args.learning_rate = 1e-5


model = QuestionAnsweringModel(
    "xlmroberta", model_name, args=model_args,use_cuda = torch.cuda.is_available()
)

# Train the model
model.train_model(train_data, eval_data=test_data)




INFO:filelock:Lock 139700173568400 acquired on /root/.cache/huggingface/transformers/884c0f33c3e77d0ae9767739e9a8d4e4bd7cad073cd0a77f39f9653a69cc0ab6.250ea9b0a06128f28a2553e930e58780d5e31f8f99bb8ef0ee7bf7996437ada6.lock


Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

INFO:filelock:Lock 139700173568400 released on /root/.cache/huggingface/transformers/884c0f33c3e77d0ae9767739e9a8d4e4bd7cad073cd0a77f39f9653a69cc0ab6.250ea9b0a06128f28a2553e930e58780d5e31f8f99bb8ef0ee7bf7996437ada6.lock
INFO:filelock:Lock 139700348076880 acquired on /root/.cache/huggingface/transformers/ced30997b1efefcbd770c0e568c87f16185dfad9179e8ea1185c25b1d6b8c020.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8.lock


Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

INFO:filelock:Lock 139700348076880 released on /root/.cache/huggingface/transformers/ced30997b1efefcbd770c0e568c87f16185dfad9179e8ea1185c25b1d6b8c020.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8.lock
INFO:filelock:Lock 139700346371472 acquired on /root/.cache/huggingface/transformers/f08358ed7cdb7d565d43304f9ace716477913b6c3266173d3ac06e398d24d282.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8.lock


...

/usr/local/lib/python3.7/dist-packages/simpletransformers/question_answering/question_answering_model.py:752: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm


Running Epoch 1 of 3:   0%|          | 0/654 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/654 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model: Training of xlmroberta model complete. Saved to outputs/.


(1962, 0.20621762382864353)

In [ ]:
result

In [ ]:
result, texts = model.eval_model(test_data)

In [ ]:
texts

# **Predict Test Set**

In [ ]:
test_set = pd.read_csv('/content/drive/MyDrive/Hackathon/Test.csv')

In [ ]:
test_set

,id,article_id,question,Unnamed: 3
0,1,220726,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN
1,2,8217,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN
2,3,35303,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN
3,4,12493,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN
4,5,11209,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN
...,...,...,...,...
5603,5604,SCG217,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,NaN
5604,5605,SCG217,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,NaN
5605,5606,SCG217,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,NaN
5606,5607,SCG217,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,NaN


In [ ]:
def readfile2(path):
  try:
    f = open("/content/wiki-documents-nsc/"+str(path)+'.txt', "r")
    text = f.read()
    text = remove(text)
    return text
  except:
    f = open("/content/drive/MyDrive/Hackathon/"+str(path)+'.txt', "r")
    text = f.read()
    text = remove(text)
    return text

In [ ]:
test_set["context_text"] = test_set['article_id'].apply(readfile2)


In [ ]:
test_set

,id,article_id,question,Unnamed: 3,context_text
0,1,220726,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,"<doc id=""220726"" url=""https://th.wikipedia.org..."
1,2,8217,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,"<doc id=""8217"" url=""https://th.wikipedia.org/w..."
2,3,35303,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,"<doc id=""35303"" url=""https://th.wikipedia.org/..."
3,4,12493,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,"<doc id=""12493"" url=""https://th.wikipedia.org/..."
4,5,11209,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,"<doc id=""11209"" url=""https://th.wikipedia.org/..."
...,...,...,...,...,...
5603,5604,SCG217,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,NaN,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5604,5605,SCG217,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,NaN,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5605,5606,SCG217,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,NaN,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5606,5607,SCG217,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,NaN,เมื่อมาทำงานจริงกับ Digitally Transforming Org...


In [ ]:
test_set["context_text"] = test_set["context_text"].apply(extract_text)

In [ ]:
test_set["context_text_wordsegged"] = test_set["context_text"].apply(lambda x: ' '.join(word_tokenize(x)))
test_set["context_text_wordsegged"] = test_set["context_text_wordsegged"].apply(lambda x: replaceSpace(x))

In [ ]:
test_set["question_wordsegged"] = test_set["question"].apply(lambda x: ' '.join(word_tokenize(x)))
test_set["question_wordsegged"] = test_set["question_wordsegged"].apply(lambda x: replaceSpace(x))

In [ ]:
test_set

,id,article_id,question,Unnamed: 3,context_text,context_text_wordsegged,question_wordsegged
0,1,220726,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,NaN,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,ประยุทธ์ จันทร์ โอชา พลเอก ประยุทธ์ จันทร์ โอช...,ใคร เป็น ผู้เรียบเรียง เนื้อ ร้องเพลง คืน ความ...
1,2,8217,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,NaN,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,แมว แมว () เป็น สัตว์เลี้ยงลูกด้วยนม อยู่ ใน ต...,การ จัด นิทรรศการ พันธุ์ แมว ในประเทศ อังกฤษ เ...
2,3,35303,ชาวอินเดียเรียกหมากรุกว่าอะไร,NaN,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,หมากรุก หมากรุก เป็น เกม กระดาน ชนิด หนึ่ง มี ...,ชาว อินเดีย เรียก หมากรุก ว่า อะไร
3,4,12493,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,NaN,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,จิ้งจก บ้าน จิ้งจก บ้าน หรือ ที่ นิยม เรียก สั...,จิ้งจก เป็น สัตว์เลื้อยคลาน ที่อยู่ ใน สกุล ใด
4,5,11209,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,NaN,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,เสือ เสือ เป็น สัตว์เลี้ยงลูกด้วยนม ใน วงศ์ ฟิ...,อะไร คือ จุดเด่น เฉพาะตัว ของ เสือ ที่ สัตว์ อ...
...,...,...,...,...,...,...,...
5603,5604,SCG217,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,NaN,เมื่อมาทำงานจริงกับ Digitally Transforming Org...,เมื่อ มา ทำงาน จริง กับ Digitally Transforming...,จาก บทความ ผู้เขียน ร่วมงาน กับ DO เป็น ระยะเว...
5604,5605,SCG217,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,NaN,เมื่อมาทำงานจริงกับ Digitally Transforming Org...,เมื่อ มา ทำงาน จริง กับ Digitally Transforming...,เรื่อง อะไร ที่ ผู้เขียน ได้ พบ ว่า เป็น ความส...
5605,5606,SCG217,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,NaN,เมื่อมาทำงานจริงกับ Digitally Transforming Org...,เมื่อ มา ทำงาน จริง กับ Digitally Transforming...,มี Project อะไร บ้าง ที่ ผู้เขียน ต้อง ทำงาน พ...
5606,5607,SCG217,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,NaN,เมื่อมาทำงานจริงกับ Digitally Transforming Org...,เมื่อ มา ทำงาน จริง กับ Digitally Transforming...,ผู้เขียน มองว่า เทคนิค ที่ สำคัญ ใน การ ร่วมงา...


In [ ]:
predict_set = []
for idx, row in test_set.iterrows():
  doc = {}
  doc["context"] = row["context_text_wordsegged"]
  doc["qas"] = [{"id":idx,"question":row["question_wordsegged"]}]
  predict_set.append(doc)

In [ ]:
predict_set

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
result = model.predict(predict_set)



INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features: 100%|██████████| 5608/5608 [02:33<00:00, 36.53it/s]

add example index and unique id: 100%|██████████| 5608/5608 [00:00<00:00, 212310.40it/s]


Running Prediction:   0%|          | 0/2474 [00:00<?, ?it/s]

In [ ]:
result[0]

[{'answer': ['พันตรี สุ ระ ชัย ถวิล ไพร',
   '',
   'ประยุทธ์ จันทร์ โอชา',
   'พันตรี สุ ระ ชัย ถวิล ไพร ขับร้อง โดย จ่า สิบเอก พงศธร พอ จิต',
   'สุ ระ ชัย ถวิล ไพร',
   'มี พันตรี สุ ระ ชัย ถวิล ไพร',
   'พันตรี สุ ระ ชัย ถวิล ไพร ขับร้อง',
   'ถวิล ไพร',
   'ประยุทธ์ จันทร์ โอชา พลเอก ประยุทธ์ จันทร์ โอชา',
   'พันตรี สุ ระ ชัย ถวิล ไพร ขับร้อง โดย จ่า สิบเอก พงศธร พอ จิต ซึ่ง เป็น ผู้ขับ ร้องเพลง คืน ความสุข ให้ ประเทศ ไทย มา ขับ ร้องเพลง นี้ ได้ เผยแพร่ เป็นครั้งแรก ให้ สื่อมวลชน ฟัง ระหว่าง รอ การแถลงข่าว ของ นายกรัฐมนตรี เมื่อ วันที่ 22 ธันวาคม 2558 ความ ยาว 4 นาที - พ.ศ. 2559 ได้ แต่งเพลง',
   'พันตรี',
   'พันตรี สุ ระ ชัย',
   'ไพร',
   'ระ ชัย ถวิล ไพร',
   'พันตรี สุ ระ ชัย ถวิล ไพร ขับร้อง โดย จ่า สิบเอก พงศธร พอ จิต ซึ่ง เป็น ผู้ขับ ร้องเพลง คืน ความสุข ให้ ประเทศ ไทย มา ขับ ร้องเพลง นี้ ได้ เผยแพร่ เป็นครั้งแรก ให้ สื่อมวลชน ฟัง ระหว่าง รอ การแถลงข่าว ของ นายกรัฐมนตรี เมื่อ วันที่ 22 ธันวาคม 2558 ความ ยาว 4 นาที - พ.ศ. 2559',
   'ประยุทธ์ จันทร์ โอชา พลเอก ประยุทธ์ จันท

# **Make Predict Better**

In [ ]:
def replaceAnswer(text):
  replaceWord = [['“ ','“']
                 ,[' ”','”']
                 ,['" ','"']
                 ,[' "','"']
                 ,['# ','#']
                 ]

  for i in range(len(replaceWord)):
    text = text.replace(replaceWord[i][0],replaceWord[i][1])

  return text

In [ ]:
def GetQuestKey(question):
  questionKey = [['กี่','sub']
                  ,['แฮชแท็ก','hasttag']
                  ,['แฮชแทก','hasttag']
                  ,['hashtag','hasttag']
                  ,['นาน','time']
                  ,['ปี','number']   
                  #,['เท่าใด','number'] 
                  #,['เท่าไร','number'] 
                  ]
  for i in range(len(questionKey)):
    if questionKey[i][0] in question:
      return questionKey[i]

  return ['','']

In [ ]:
def GetAnswerKey(answer):
  answerKey = [
               ['http','url']
               ,['https','url']
                  ]
  for i in range(len(answerKey)):
    if answerKey[i][0] in answer:
      return answerKey[i]

  return ['','']

In [ ]:
def extract_hashtags(question,context):

    hashtagQuestion=[]
    hashtagQuestion= re.findall(r"#([ก-๙A-Za-z0-9]+)", question)
    
    tmpContext = context
    tmpContext = tmpContext.replace('-',' ')

    hashtagAnswer = []
    hashtagAnswer= re.findall(r"#([ก-๙A-Za-z0-9]+)", tmpContext)

    tmpStr =""
    for i in range(len(hashtagAnswer)):
      if hashtagAnswer[i] not in hashtagQuestion:
        tmpStr += "#"+hashtagAnswer[i]+" "

    tmpStr = tmpStr.strip()

    return tmpStr

In [ ]:
def SelectAnswer(question,ans,context):  
  resultAnswerIndex = 0
  newAnswer = ans[resultAnswerIndex]
  answerTimeKey = ['วินาที','นาที','ชั่วโมง','วัน','สัปดาห์','เดือน','ปี']

  typeAnswer = GetAnswerKey(ans)
  typeQuestion = GetQuestKey(question)
  foundInAnswer = False
  
  if typeAnswer[1]=='':
    pass
  elif typeAnswer[1]=='url':    
    newAnswer =  concatURL(ans[resultAnswerIndex],stopAtLenght=True)
    return newAnswer

  if typeQuestion[1]=='':
    pass
  elif typeQuestion[1]=='sub':
    tokenQ = word_tokenize(question)
    for iToken in range(len(tokenQ)):
      keyNoun = ""
      if tokenQ[iToken]==typeQuestion[0] and iToken+1 < (len(tokenQ)-1):
        keyNoun=tokenQ[iToken+1]
        break
    if keyNoun!="":
      foundKeyNoun = False
      for iAnswer in range(len(ans)):
        tokenA = word_tokenize(ans[iAnswer])
        if keyNoun in tokenA:
          #Update Answer
          resultAnswerIndex=iAnswer
          foundKeyNoun=True        
        if foundKeyNoun:
          break
    newAnswer = ans[resultAnswerIndex]
  elif typeQuestion[1]=='hasttag':
    newAnswer = extract_hashtags(question,context)
  elif typeQuestion[1]=='time':    
    for i in range(len(answerTimeKey)):
      if answerTimeKey[i] in ans:
        foundInAnswer = True
        break
     #ถ้าในคำตอบไม่เกี่ยวกับเวลา ต้องหาคำตอบใหม่
    if not foundInAnswer:
      for i in range(len(answerTimeKey)):
        tmpWord = answerTimeKey[i]
        try:
          indexWord = context.index(tmpWord)
          if indexWord>0 :
            tmpSentence = context[:indexWord+len(tmpWord)]
            tokenAns = word_tokenize(tmpSentence)            
            if len(tokenAns)>=3:
              if (tokenAns[len(tokenAns)-2]).isnumeric() or (tokenAns[len(tokenAns)-3]).isnumeric() :
                newAnswer = tokenAns[len(tokenAns)-3] + tokenAns[len(tokenAns)-2] + tokenAns[len(tokenAns)-1]
                break;

        except:
          pass
          
    
  return newAnswer

In [ ]:
def CheckSpaceSentenct(context,ans):
  
  if len(ans)==0 :
    return ans

  tmpContext=context
  newAns = ans

  if ' ' in ans :

    splitAns = ans.split(' ')
    FirstWord = splitAns[0]    
    LastWord = splitAns[len(splitAns)-1]

    FirstWordToken = word_tokenize(FirstWord)
    LastWordToken = word_tokenize(LastWord)

    FirstWord =FirstWordToken[0]
    LastWord =LastWordToken[len(LastWordToken)-1]

    FirstWordIndex = -1
    LastWordIndex = -1

    try:
      FirstWordIndex= tmpContext.index(FirstWord)
      tmpContext = tmpContext[FirstWordIndex:]

      tmpReplaceWord ='|_|'

      countSameLastWord=1
      try:
        countSameLastWord = re.findall(LastWord,tmpContext)
      except Exception as e:        
        countSameLastWord=1
            
      if len(countSameLastWord)>1 :        
        if len(LastWord)==1:
          for i in range(2):
            LastWordIndex = tmpContext.index(LastWord)            
            #แยกหน้าหลัง
            frontContext = tmpContext[:LastWordIndex+len(LastWord)]
            
            if len(frontContext)==len(tmpContext):
              backContext=''
            else:
              backContext = tmpContext[LastWordIndex+len(LastWord):]

            frontContext = frontContext.replace(LastWord,tmpReplaceWord)          
            tmpContext =frontContext+backContext
            

      FirstWordIndex= tmpContext.index(FirstWord)
      LastWordIndex= tmpContext.index(LastWord)

                        
      if FirstWordIndex>-1 and LastWordIndex>-1:
        newAns = tmpContext[FirstWordIndex: LastWordIndex + len(LastWord)]        
        if len(countSameLastWord)>1:
          newAns = newAns.replace(tmpReplaceWord,LastWord)

        return newAns

    except Exception as e:
      return newAns

  return newAns

In [ ]:
test_set['answer'] = ''

In [ ]:
pdResult = pd.DataFrame(result[0])
pdProb = pd.DataFrame(result[1])
pdResult['probability']= pdProb['probability']
pdResult['question'] = test_set['question']
pdResult['keyanswer'] = test_set['answer']
pdResult['context'] = test_set['context_text']

In [ ]:
pdResult

,id,answer,probability,question,keyanswer,context
0,0,"[พันตรี สุ ระ ชัย ถวิล ไพร, , ประยุทธ์ จันทร์ ...","[0.6594709658344665, 0.31889932159213846, 0.01...",ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...
1,1,"[, เสือ ไซบีเรีย, แมว แมว () เป็น สัตว์เลี้ยงล...","[0.9998855357756457, 9.248680716131916e-05, 1....",การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...
2,2,"[, 2, 2 ฝ่าย, 4 พวก นั้น ให้ เป็น แต่ 2 พวก ตั...","[0.9999999989965977, 2.878015312358801e-10, 1....",ชาวอินเดียเรียกหมากรุกว่าอะไร,,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...
3,3,"[Hemidactylus, สกุล Hemidactylus, จิ้งจก บ้าน ...","[0.9990074659679036, 0.0009546955855157108, 1....",จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...
4,4,"[คำราม ได้, คำราม, ไม่ สามารถ คำราม ได้, สามาร...","[0.7928625838956046, 0.09750920611441403, 0.05...",อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...
...,...,...,...,...,...,...
5603,5603,"[3 เดือน 20 วัน, 3 เดือน, 3, 3 เดือน 20 วัน Ph...","[0.8400663251084263, 0.10045416464757889, 0.04...",จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5604,5604,"[User Experience design, User Experience desig...","[0.5177325896664927, 0.4560058998957894, 0.009...",เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5605,5605,"[, Digitally Transforming Organization บทความ ...","[0.9999999998419073, 2.8944960583868805e-11, 1...",มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5606,5606,"[, Digitally Transforming Organization บทความ ...","[0.9999999999368849, 5.502705420121153e-12, 5....",ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,,เมื่อมาทำงานจริงกับ Digitally Transforming Org...


In [ ]:
listPredicted = []
for index in range(len(pdResult)):    
  select = SelectAnswer(pdResult.loc[index]['question'],pdResult.loc[index]['answer'],pdResult.loc[index]['context'])
  repla = replaceAnswer(select)  
  checkspace = CheckSpaceSentenct(pdResult.loc[index]['context'] ,repla)  
  listPredicted.append(checkspace)
pdResult['predicted']=listPredicted

In [ ]:
pdResult

,id,answer,probability,question,keyanswer,context,predicted
0,0,"[พันตรี สุ ระ ชัย ถวิล ไพร, , ประยุทธ์ จันทร์ ...","[0.6594709658344665, 0.31889932159213846, 0.01...",ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...,พันตรี สุระชัย ถวิลไพร
1,1,"[, เสือ ไซบีเรีย, แมว แมว () เป็น สัตว์เลี้ยงล...","[0.9998855357756457, 9.248680716131916e-05, 1....",การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,,แมว แมว () เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระก...,
2,2,"[, 2, 2 ฝ่าย, 4 พวก นั้น ให้ เป็น แต่ 2 พวก ตั...","[0.9999999989965977, 2.878015312358801e-10, 1....",ชาวอินเดียเรียกหมากรุกว่าอะไร,,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...,
3,3,"[Hemidactylus, สกุล Hemidactylus, จิ้งจก บ้าน ...","[0.9990074659679036, 0.0009546955855157108, 1....",จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...,Hemidactylus
4,4,"[คำราม ได้, คำราม, ไม่ สามารถ คำราม ได้, สามาร...","[0.7928625838956046, 0.09750920611441403, 0.05...",อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...,คำรามได้
...,...,...,...,...,...,...,...
5603,5603,"[3 เดือน 20 วัน, 3 เดือน, 3, 3 เดือน 20 วัน Ph...","[0.8400663251084263, 0.10045416464757889, 0.04...",จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,,เมื่อมาทำงานจริงกับ Digitally Transforming Org...,3 เดือน 20 วัน
5604,5604,"[User Experience design, User Experience desig...","[0.5177325896664927, 0.4560058998957894, 0.009...",เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,,เมื่อมาทำงานจริงกับ Digitally Transforming Org...,User Experience design
5605,5605,"[, Digitally Transforming Organization บทความ ...","[0.9999999998419073, 2.8944960583868805e-11, 1...",มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,,เมื่อมาทำงานจริงกับ Digitally Transforming Org...,
5606,5606,"[, Digitally Transforming Organization บทความ ...","[0.9999999999368849, 5.502705420121153e-12, 5....",ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,,เมื่อมาทำงานจริงกับ Digitally Transforming Org...,


In [ ]:
print(pdResult['predicted'][0][:10])

พันตรี สุร


In [ ]:
count = 0
for i in range(len(pdResult['predicted'])):
   if len(pdResult['predicted'][i]) == 0:
      count = count+1
      print('1')
      #pdResult['predicted'][i] = 'ใช่'
   elif len(pdResult['predicted'][i]) > 100:
      pdResult['predicted'][i] = pdResult['predicted'][0][:100]


In [ ]:
count

2232

In [ ]:
maxID = 5608
doAnswer = 5608
id = []
for i in range(1,maxID+1):
  id.append(i)

predicted = []
for i in range(0,doAnswer):
  predicted.append(pdResult.loc[i]['predicted'])

if doAnswer<maxID:
  for i in range(doAnswer,maxID):
    predicted.append('')


In [ ]:
pdResult.loc[3]

id                                                             3
answer         [Hemidactylus, สกุล Hemidactylus, จิ้งจก บ้าน ...
probability    [0.9990074659679036, 0.0009546955855157108, 1....
question                จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด
keyanswer                                                       
context        จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...
predicted                                           Hemidactylus
Name: 3, dtype: object

In [ ]:
dataKag = {'Id':id , 'answer': predicted}
dataKaggle = pd.DataFrame(data=dataKag)

In [ ]:
dataKaggle['answer']

0       พันตรี สุระชัย ถวิลไพร
1                             
2                             
3                 Hemidactylus
4                     คำรามได้
                 ...          
5603            3 เดือน 20 วัน
5604    User Experience design
5605                          
5606                          
5607                          
Name: answer, Length: 5608, dtype: object

In [ ]:
dataKaggle.to_csv(r'/content/drive/MyDrive/Hackathon/submission2.csv',index=False)